## drf-access-policy

drf-access-policy を使ってアクセス制御をする.

https://github.com/rsinger86/drf-access-policy

## 削除機能の制限を確認

記事の author のユーザーのみ, 記事を削除できることを確認する.

In [4]:
from rest_framework.test import APIClient
client = APIClient()

In [5]:
## editor グループを作成
editor_group,_ = Group.objects.get_or_create(name='editor')

In [6]:
author = User.objects.get(pk=1)
another = User.objects.get(pk=2)
reader = User.objects.get(pk=3)

# ユーザー を editor group に登録
editor_group.user_set.add(author)
editor_group.user_set.add(another)

In [7]:
## status でフィルタリングされていることを確認

In [8]:
Article.objects.create(author=author, title='unpublish article', body='spam', status='unpublish')
Article.objects.create(author=author, title='published article', body='ham', status='publish')

<Article: 54 - published article>

In [13]:
client.force_authenticate(reader)
url = reverse('article-list')
client.get(url).json()

{'count': 1,
 'next': None,
 'previous': None,
 'results': [{'id': 54,
   'category': None,
   'title': 'published article',
   'body': 'ham',
   'status': 'publish',
   'author': 1,
   'tags': []}]}

In [18]:
client.force_authenticate(author)
client.get(url).json()

{'count': 2,
 'next': None,
 'previous': None,
 'results': [{'id': 53,
   'category': None,
   'title': 'unpublish article',
   'body': 'spam',
   'status': 'unpublish',
   'author': 1,
   'tags': []},
  {'id': 54,
   'category': None,
   'title': 'published article',
   'body': 'ham',
   'status': 'publish',
   'author': 1,
   'tags': []}]}

テスト用の記事を登録

In [19]:
article = Article(author=author, title='author-test', body='article body')
article.save()

### 他のeditorが削除できないことを確認する

In [20]:
url = reverse('article-detail', kwargs={'pk': article.id})
client.force_authenticate(another)

In [21]:
res = client.delete(url)
res.status_code

Forbidden: /articles/48/


403

### author が削除できることを確認

In [22]:
client.force_authenticate(author)

In [23]:
res = client.delete(url)
res.status_code

204